In [1]:
# =========================================================
# Notebook 1 (V3): NEW TABLE (Front + Back Pflicht) -> DB
# - Neue Tabelle: pokemon_card_samples_fb
# - Label manuell
# - Randtreuer Warp + Debug Quad
# - Multi-View Preview je Seite
# =========================================================

# Schritt 1: Abhängigkeiten installieren
%pip install --quiet psycopg2-binary gradio pillow opencv-python-headless python-dotenv numpy

# Schritt 2: Imports & Settings
import os, io, uuid, json, hashlib, socket, traceback
import numpy as np
import cv2
from PIL import Image, ImageOps

import psycopg2
import psycopg2.extras
import gradio as gr
from dotenv import load_dotenv
load_dotenv()

TABLE = "pokemon_card_samples_fb"

LABELS = ["NM", "EX", "GD", "LP", "PL", "PO"]

IMG_H = 352
IMG_W = 256

# Expand getrennt, weil Back oft blau-maskiert gut expandiert, Front manchmal minimal weniger
QUAD_EXPAND_BACK  = 1.04
QUAD_EXPAND_FRONT = 1.03

CORNER_RADIUS_PX = int(round(IMG_W * 0.05))
APPLY_ROUNDED_MASK_TO_IMAGE = False

CROP_FRAC = 0.62


# Schritt 3: DB Verbindung
def get_conn():
    return psycopg2.connect(
        host=os.getenv("PGHOST", "localhost"),
        port=int(os.getenv("PGPORT", "5434")),
        dbname=os.getenv("PGDATABASE", "sam1988"),
        user=os.getenv("PGUSER", "sam1988"),
        password=os.getenv("PGPASSWORD", "Ss190488!")  # besser per ENV setzen
    )

    return psycopg2.connect(host=host, port=port, dbname=db, user=user, password=pw)


# Schritt 4: Neue Tabelle anlegen
def ensure_schema():
    labels_sql = ",".join([f"'{l}'" for l in LABELS])

    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(f"""
            CREATE TABLE IF NOT EXISTS {TABLE} (
                id UUID PRIMARY KEY,
                created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),

                label TEXT NOT NULL CHECK (label IN ({labels_sql})),
                note TEXT,

                -- BACK (Pflicht)
                back_raw_sha256 TEXT NOT NULL UNIQUE,
                back_raw_format TEXT NOT NULL,
                back_raw_w INT,
                back_raw_h INT,
                back_raw_bytes BYTEA NOT NULL,

                back_proc_format TEXT NOT NULL,
                back_proc_w INT NOT NULL,
                back_proc_h INT NOT NULL,
                back_proc_bytes BYTEA NOT NULL,

                back_proc_mask_format TEXT NOT NULL DEFAULT 'png',
                back_proc_mask_w INT NOT NULL DEFAULT {IMG_W},
                back_proc_mask_h INT NOT NULL DEFAULT {IMG_H},
                back_proc_mask_bytes BYTEA,

                back_proc_method TEXT,
                back_proc_quad_expand REAL,

                -- FRONT (Pflicht)
                front_raw_sha256 TEXT NOT NULL UNIQUE,
                front_raw_format TEXT NOT NULL,
                front_raw_w INT,
                front_raw_h INT,
                front_raw_bytes BYTEA NOT NULL,

                front_proc_format TEXT NOT NULL,
                front_proc_w INT NOT NULL,
                front_proc_h INT NOT NULL,
                front_proc_bytes BYTEA NOT NULL,

                front_proc_mask_format TEXT NOT NULL DEFAULT 'png',
                front_proc_mask_w INT NOT NULL DEFAULT {IMG_W},
                front_proc_mask_h INT NOT NULL DEFAULT {IMG_H},
                front_proc_mask_bytes BYTEA,

                front_proc_method TEXT,
                front_proc_quad_expand REAL
            );
            """)

            cur.execute(f"CREATE INDEX IF NOT EXISTS idx_{TABLE}_label ON {TABLE}(label);")
            cur.execute(f"CREATE INDEX IF NOT EXISTS idx_{TABLE}_created_at ON {TABLE}(created_at DESC);")

def db_counts():
    with get_conn() as conn:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(f"""
                SELECT label, COUNT(*)::int AS n
                FROM {TABLE}
                GROUP BY label
                ORDER BY label;
            """)
            rows = cur.fetchall()

    counts = {r["label"]: r["n"] for r in rows}
    for l in LABELS:
        counts.setdefault(l, 0)
    counts["TOTAL"] = sum(counts[l] for l in LABELS)
    return counts

ensure_schema()
print(f"✅ Schema OK ({TABLE}). Counts:", db_counts())


# Schritt 5: Preprocessing
def pil_to_jpeg_bytes(pil_img: Image.Image, quality: int = 92) -> bytes:
    pil_img = ImageOps.exif_transpose(pil_img).convert("RGB")
    buf = io.BytesIO()
    pil_img.save(buf, format="JPEG", quality=quality, optimize=True)
    return buf.getvalue()

def bgr_from_pil(pil_img: Image.Image) -> np.ndarray:
    rgb = np.array(ImageOps.exif_transpose(pil_img).convert("RGB"))
    return cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

def pil_from_bgr(bgr: np.ndarray) -> Image.Image:
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(rgb)

def order_points(pts):
    pts = np.array(pts, dtype=np.float32)
    rect = np.zeros((4, 2), dtype=np.float32)
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]      # tl
    rect[2] = pts[np.argmax(s)]      # br
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]   # tr
    rect[3] = pts[np.argmax(diff)]   # bl
    return rect

def expand_quad(quad: np.ndarray, scale: float) -> np.ndarray:
    q = quad.astype(np.float32)
    c = q.mean(axis=0, keepdims=True)
    return (c + (q - c) * scale).astype(np.float32)

def warp_quad(bgr, quad, out_w=IMG_W, out_h=IMG_H):
    rect = order_points(quad)
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype=np.float32)
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(
        bgr, M, (out_w, out_h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT
    )

def overlay_quad_debug(bgr, quad):
    dbg = bgr.copy()
    q = order_points(quad).astype(int)
    cv2.polylines(dbg, [q], isClosed=True, color=(0, 255, 0), thickness=4)
    for (x, y) in q:
        cv2.circle(dbg, (x, y), 10, (0, 0, 255), -1)
    return dbg

# --- Extraction Methoden ---
def try_extract_by_blue_mask(bgr):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    lower = np.array([80, 40, 40], dtype=np.uint8)
    upper = np.array([150, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  k, iterations=1)

    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None

    cnt = max(cnts, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    if area < 0.10 * (bgr.shape[0] * bgr.shape[1]):
        return None

    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    if len(approx) == 4:
        return approx.reshape(-1, 2)

    rect = cv2.minAreaRect(cnt)
    return cv2.boxPoints(rect)

def try_extract_by_edges(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150)
    edges = cv2.dilate(edges, None, iterations=2)

    cnts, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None

    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        if area < 0.10 * (bgr.shape[0] * bgr.shape[1]):
            continue
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
        if len(approx) == 4:
            return approx.reshape(-1, 2)
    return None

def try_extract_by_adaptive_thresh(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    th = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,
        31, 7
    )
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, k, iterations=2)

    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None

    cnt = max(cnts, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    if area < 0.10 * (bgr.shape[0] * bgr.shape[1]):
        return None

    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    if len(approx) == 4:
        return approx.reshape(-1, 2)

    rect = cv2.minAreaRect(cnt)
    return cv2.boxPoints(rect)

def normalize_to_target(bgr, side: str, out_w=IMG_W, out_h=IMG_H):
    """
    side: "back" oder "front"
    back:  blue_mask -> edges -> adaptive
    front: edges -> adaptive -> blue_mask (letzte Option)
    """
    if side == "back":
        candidates = [("blue_mask", try_extract_by_blue_mask), ("edges", try_extract_by_edges), ("adaptive", try_extract_by_adaptive_thresh)]
        quad_expand = QUAD_EXPAND_BACK
    else:
        candidates = [("edges", try_extract_by_edges), ("adaptive", try_extract_by_adaptive_thresh), ("blue_mask", try_extract_by_blue_mask)]
        quad_expand = QUAD_EXPAND_FRONT

    quad = None
    method = None
    for nm, fn in candidates:
        quad = fn(bgr)
        if quad is not None:
            method = nm
            break

    if quad is None:
        resized = cv2.resize(bgr, (out_w, out_h), interpolation=cv2.INTER_AREA)
        return resized, "fallback_resize", False, None, quad_expand

    quad = expand_quad(np.array(quad), scale=quad_expand)
    dbg = overlay_quad_debug(bgr, quad)
    warped = warp_quad(bgr, quad, out_w=out_w, out_h=out_h)

    if warped.shape[1] > warped.shape[0]:
        warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)

    warped = cv2.resize(warped, (out_w, out_h), interpolation=cv2.INTER_AREA)
    return warped, f"{method}_expand{quad_expand}", True, dbg, quad_expand

def encode_png_bytes_from_bgr(bgr):
    ok, enc = cv2.imencode(".png", bgr)
    if not ok:
        raise RuntimeError("PNG encoding failed")
    return enc.tobytes()

def encode_png_bytes_gray(gray: np.ndarray) -> bytes:
    ok, enc = cv2.imencode(".png", gray)
    if not ok:
        raise RuntimeError("PNG encoding failed (mask)")
    return enc.tobytes()

def rounded_rect_mask(h: int, w: int, r: int) -> np.ndarray:
    r = int(max(0, r))
    r = min(r, min(h, w) // 2)
    mask = np.zeros((h, w), dtype=np.uint8)
    if r == 0:
        mask[:] = 255
        return mask

    cv2.rectangle(mask, (r, 0), (w - r - 1, h - 1), 255, -1)
    cv2.rectangle(mask, (0, r), (w - 1, h - r - 1), 255, -1)
    cv2.circle(mask, (r, r), r, 255, -1)
    cv2.circle(mask, (w - r - 1, r), r, 255, -1)
    cv2.circle(mask, (r, h - r - 1), r, 255, -1)
    cv2.circle(mask, (w - r - 1, h - r - 1), r, 255, -1)
    return mask

def make_multiviews(img_rgb_uint8: np.ndarray):
    img = img_rgb_uint8
    H, W = img.shape[:2]
    ch = int(round(H * CROP_FRAC))
    cw = int(round(W * CROP_FRAC))

    def crop(y0, x0, y1, x1):
        c = img[y0:y1, x0:x1]
        return cv2.resize(c, (IMG_W, IMG_H), interpolation=cv2.INTER_AREA)

    full = img
    tl = crop(0, 0, ch, cw)
    tr = crop(0, W-cw, ch, W)
    bl = crop(H-ch, 0, H, cw)
    br = crop(H-ch, W-cw, H, W)

    y_mid0 = (H - ch)//2
    x_mid0 = (W - cw)//2
    top = crop(0, x_mid0, ch, x_mid0+cw)
    bottom = crop(H-ch, x_mid0, H, x_mid0+cw)
    left = crop(y_mid0, 0, y_mid0+ch, cw)
    right = crop(y_mid0, W-cw, y_mid0+ch, W)

    base_views = [
        ("full", full),
        ("corner_tl", tl), ("corner_tr", tr), ("corner_bl", bl), ("corner_br", br),
        ("edge_top", top), ("edge_bottom", bottom), ("edge_left", left), ("edge_right", right),
    ]

    def aug(name, v):
        return [
            (name, v),
            (name + "_hflip", cv2.flip(v, 1)),
            (name + "_vflip", cv2.flip(v, 0)),
            (name + "_rot180", cv2.rotate(v, cv2.ROTATE_180)),
        ]

    views_named = []
    views_named.extend(base_views)            # 9
    views_named.extend(aug("full", full)[1:]) # +3
    for nm, v in [("corner_tl", tl), ("corner_tr", tr), ("corner_bl", bl), ("corner_br", br)]:
        views_named.extend(aug(nm, v)[1:])    # +12
    return views_named                         # total 24

def prepare_side_for_db(pil_img: Image.Image, side: str):
    raw_jpeg = pil_to_jpeg_bytes(pil_img, quality=92)
    raw_pil = Image.open(io.BytesIO(raw_jpeg)).convert("RGB")
    raw_w, raw_h = raw_pil.size

    bgr = bgr_from_pil(raw_pil)
    proc_bgr, method, extracted, dbg_bgr, quad_expand = normalize_to_target(bgr, side=side, out_w=IMG_W, out_h=IMG_H)

    mask = rounded_rect_mask(IMG_H, IMG_W, CORNER_RADIUS_PX)
    if APPLY_ROUNDED_MASK_TO_IMAGE:
        proc_bgr = cv2.bitwise_and(proc_bgr, proc_bgr, mask=mask)

    proc_png = encode_png_bytes_from_bgr(proc_bgr)
    mask_png = encode_png_bytes_gray(mask)

    raw_preview_pil = raw_pil
    dbg_preview_pil = pil_from_bgr(dbg_bgr) if dbg_bgr is not None else None
    proc_preview_pil = pil_from_bgr(proc_bgr)
    mask_preview_pil = Image.fromarray(mask)

    proc_rgb = cv2.cvtColor(proc_bgr, cv2.COLOR_BGR2RGB)
    views_named = make_multiviews(proc_rgb)
    views_gallery = [(Image.fromarray(v), name) for name, v in views_named]

    info = {
        "side": side,
        "raw_size": [raw_w, raw_h],
        "proc_size": [IMG_W, IMG_H],
        "method": method,
        "extracted": extracted,
        "quad_expand": float(quad_expand),
        "views_count": len(views_named),
        "raw_sha256": hashlib.sha256(raw_jpeg).hexdigest()
    }

    return (
        raw_jpeg, "jpeg", raw_w, raw_h,
        proc_png, "png", IMG_W, IMG_H,
        mask_png, "png", IMG_W, IMG_H,
        raw_preview_pil, dbg_preview_pil, proc_preview_pil, mask_preview_pil,
        views_gallery,
        info
    )


# Schritt 6: DB Insert/Fetch
def fmt_pg_error(e: Exception) -> str:
    if isinstance(e, psycopg2.Error):
        parts = [f"{type(e).__name__}: {e}"]
        if getattr(e, "pgcode", None):
            parts.append(f"pgcode: {e.pgcode}")
        if getattr(e, "pgerror", None):
            parts.append(f"pgerror: {e.pgerror}")
        diag = getattr(e, "diag", None)
        if diag is not None:
            for k in ["message_detail", "message_hint", "schema_name", "table_name", "column_name", "constraint_name"]:
                v = getattr(diag, k, None)
                if v:
                    parts.append(f"{k}: {v}")
        return "\n".join(parts)
    return f"{type(e).__name__}: {e}"

def insert_sample_front_back(
    label, note,
    back_raw_bytes, back_raw_format, back_raw_w, back_raw_h,
    back_proc_bytes, back_proc_format, back_proc_w, back_proc_h,
    back_mask_bytes, back_mask_format, back_mask_w, back_mask_h,
    back_proc_method, back_proc_quad_expand,
    front_raw_bytes, front_raw_format, front_raw_w, front_raw_h,
    front_proc_bytes, front_proc_format, front_proc_w, front_proc_h,
    front_mask_bytes, front_mask_format, front_mask_w, front_mask_h,
    front_proc_method, front_proc_quad_expand,
):
    back_sha  = hashlib.sha256(back_raw_bytes).hexdigest()
    front_sha = hashlib.sha256(front_raw_bytes).hexdigest()
    sample_id = uuid.uuid4()

    try:
        with get_conn() as conn:
            with conn.cursor() as cur:
                cur.execute(f"""
                    INSERT INTO {TABLE} (
                        id, label, note,

                        back_raw_sha256, back_raw_format, back_raw_w, back_raw_h, back_raw_bytes,
                        back_proc_format, back_proc_w, back_proc_h, back_proc_bytes,
                        back_proc_mask_format, back_proc_mask_w, back_proc_mask_h, back_proc_mask_bytes,
                        back_proc_method, back_proc_quad_expand,

                        front_raw_sha256, front_raw_format, front_raw_w, front_raw_h, front_raw_bytes,
                        front_proc_format, front_proc_w, front_proc_h, front_proc_bytes,
                        front_proc_mask_format, front_proc_mask_w, front_proc_mask_h, front_proc_mask_bytes,
                        front_proc_method, front_proc_quad_expand
                    )
                    VALUES (
                        %s,%s,%s,
                        %s,%s,%s,%s,%s,
                        %s,%s,%s,%s,
                        %s,%s,%s,%s,
                        %s,%s,
                        %s,%s,%s,%s,%s,
                        %s,%s,%s,%s,
                        %s,%s,%s,%s,
                        %s,%s
                    )
                """, (
                    str(sample_id), label, note,

                    back_sha, back_raw_format, back_raw_w, back_raw_h, psycopg2.Binary(back_raw_bytes),
                    back_proc_format, back_proc_w, back_proc_h, psycopg2.Binary(back_proc_bytes),
                    back_mask_format, back_mask_w, back_mask_h, psycopg2.Binary(back_mask_bytes),
                    back_proc_method, float(back_proc_quad_expand) if back_proc_quad_expand is not None else None,

                    front_sha, front_raw_format, front_raw_w, front_raw_h, psycopg2.Binary(front_raw_bytes),
                    front_proc_format, front_proc_w, front_proc_h, psycopg2.Binary(front_proc_bytes),
                    front_mask_format, front_mask_w, front_mask_h, psycopg2.Binary(front_mask_bytes),
                    front_proc_method, float(front_proc_quad_expand) if front_proc_quad_expand is not None else None,
                ))
        return True, back_sha, front_sha, "✅ Gespeichert (Front + Back)."
    except Exception as e:
        return False, back_sha, front_sha, "❌ DB-Fehler:\n" + fmt_pg_error(e) + "\n\n" + traceback.format_exc()

def fetch_recent(limit=24, label=None):
    q = f"""
        SELECT id, created_at, label, note,
               back_proc_bytes, back_proc_method,
               front_proc_bytes, front_proc_method
        FROM {TABLE}
    """
    params = []
    if label and label != "ALL":
        q += " WHERE label = %s"
        params.append(label)
    q += " ORDER BY created_at DESC LIMIT %s"
    params.append(int(limit))

    with get_conn() as conn:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(q, params)
            return cur.fetchall()


# Schritt 7: Gradio UI
def save_from_ui(back_pil_img, front_pil_img, label, note):
    if back_pil_img is None or front_pil_img is None:
        return "❌ Bitte ZWINGEND Rückseite UND Vorderseite hochladen.", None, None, None, None, [], None, None, None, None, [], db_counts()

    if label not in LABELS:
        return f"❌ Ungültiges Label: {label}", None, None, None, None, [], None, None, None, None, [], db_counts()

    # BACK
    (b_raw_bytes, b_raw_fmt, b_raw_w, b_raw_h,
     b_proc_bytes, b_proc_fmt, b_proc_w, b_proc_h,
     b_mask_bytes, b_mask_fmt, b_mask_w, b_mask_h,
     b_raw_prev, b_dbg_prev, b_proc_prev, b_mask_prev,
     b_views_gallery,
     b_info) = prepare_side_for_db(back_pil_img, side="back")

    # FRONT
    (f_raw_bytes, f_raw_fmt, f_raw_w, f_raw_h,
     f_proc_bytes, f_proc_fmt, f_proc_w, f_proc_h,
     f_mask_bytes, f_mask_fmt, f_mask_w, f_mask_h,
     f_raw_prev, f_dbg_prev, f_proc_prev, f_mask_prev,
     f_views_gallery,
     f_info) = prepare_side_for_db(front_pil_img, side="front")

    ok, back_sha, front_sha, msg = insert_sample_front_back(
        label=label,
        note=(note or "").strip() or None,

        back_raw_bytes=b_raw_bytes, back_raw_format=b_raw_fmt, back_raw_w=b_raw_w, back_raw_h=b_raw_h,
        back_proc_bytes=b_proc_bytes, back_proc_format=b_proc_fmt, back_proc_w=b_proc_w, back_proc_h=b_proc_h,
        back_mask_bytes=b_mask_bytes, back_mask_format=b_mask_fmt, back_mask_w=b_mask_w, back_mask_h=b_mask_h,
        back_proc_method=b_info.get("method"),
        back_proc_quad_expand=b_info.get("quad_expand"),

        front_raw_bytes=f_raw_bytes, front_raw_format=f_raw_fmt, front_raw_w=f_raw_w, front_raw_h=f_raw_h,
        front_proc_bytes=f_proc_bytes, front_proc_format=f_proc_fmt, front_proc_w=f_proc_w, front_proc_h=f_proc_h,
        front_mask_bytes=f_mask_bytes, front_mask_format=f_mask_fmt, front_mask_w=f_mask_w, front_mask_h=f_mask_h,
        front_proc_method=f_info.get("method"),
        front_proc_quad_expand=f_info.get("quad_expand"),
    )

    status = {
        "status": "saved" if ok else "failed",
        "message": msg,
        "label": label,
        "back_raw_sha256": back_sha,
        "front_raw_sha256": front_sha,
        "prep_back": b_info,
        "prep_front": f_info
    }

    return (
        json.dumps(status, ensure_ascii=False, indent=2),

        # Back previews
        b_raw_prev, b_dbg_prev, b_proc_prev, b_mask_prev, b_views_gallery,

        # Front previews
        f_raw_prev, f_dbg_prev, f_proc_prev, f_mask_prev, f_views_gallery,

        db_counts()
    )

def load_gallery(label, limit, side):
    rows = fetch_recent(limit=limit, label=label)
    items = []
    for r in rows:
        if side == "front":
            proc_bytes = r.get("front_proc_bytes")
            pm = r.get("front_proc_method") or "n/a"
        else:
            proc_bytes = r.get("back_proc_bytes")
            pm = r.get("back_proc_method") or "n/a"

        if proc_bytes is None:
            continue

        pil = Image.open(io.BytesIO(bytes(proc_bytes))).convert("RGB")
        cap = f'{r["label"]} | {side} | {pm} | {r["created_at"].strftime("%Y-%m-%d %H:%M")} | {str(r["id"])[:8]}'
        items.append((pil, cap))
    return items


with gr.Blocks(title="Pokemon Card Front+Back Uploader (NEW TABLE)") as app:
    gr.Markdown(f"# 📸 Notebook 1 (V3): Upload & Label (Pflicht: Rückseite + Vorderseite)\n**Tabelle:** `{TABLE}`")

    with gr.Tab("Upload"):
        with gr.Row():
            back_in  = gr.Image(label="Rückseite hochladen", type="pil")
            front_in = gr.Image(label="Vorderseite hochladen", type="pil")

        with gr.Row():
            dd = gr.Dropdown(choices=LABELS, value="NM", label="Zustandsklasse")
            note = gr.Textbox(label="Notiz (optional)", placeholder="z.B. 'gutes Licht', 'schief', ...")

        btn = gr.Button("In DB speichern", variant="primary")
        out = gr.Code(label="Status (JSON)", language="json")
        stats = gr.JSON(label="DB Counts")

        gr.Markdown("## Rückseite: Raw / Debug / Proc / Mask + Multi-Views")
        with gr.Row():
            b_raw_prev = gr.Image(label="Back: Raw", type="pil")
            b_dbg_prev = gr.Image(label="Back: Debug Quad", type="pil")
        with gr.Row():
            b_proc_prev = gr.Image(label="Back: Proc", type="pil")
            b_mask_prev = gr.Image(label="Back: Mask", type="pil")
        b_views_gal = gr.Gallery(label="Back: Multi-Views (24)", columns=6, height="auto")

        gr.Markdown("## Vorderseite: Raw / Debug / Proc / Mask + Multi-Views")
        with gr.Row():
            f_raw_prev = gr.Image(label="Front: Raw", type="pil")
            f_dbg_prev = gr.Image(label="Front: Debug Quad", type="pil")
        with gr.Row():
            f_proc_prev = gr.Image(label="Front: Proc", type="pil")
            f_mask_prev = gr.Image(label="Front: Mask", type="pil")
        f_views_gal = gr.Gallery(label="Front: Multi-Views (24)", columns=6, height="auto")

        btn.click(
            fn=save_from_ui,
            inputs=[back_in, front_in, dd, note],
            outputs=[
                out,
                b_raw_prev, b_dbg_prev, b_proc_prev, b_mask_prev, b_views_gal,
                f_raw_prev, f_dbg_prev, f_proc_prev, f_mask_prev, f_views_gal,
                stats
            ]
        )
        stats.value = db_counts()

    with gr.Tab("Browse"):
        with gr.Row():
            gal_label = gr.Dropdown(choices=["ALL"] + LABELS, value="ALL", label="Filter (Label)")
            gal_side  = gr.Radio(choices=["back", "front"], value="back", label="Seite")
            gal_limit = gr.Slider(6, 60, value=24, step=1, label="Anzahl")
            gal_btn = gr.Button("Aktualisieren")

        gallery = gr.Gallery(label="Letzte Uploads", columns=4, height="auto")
        gal_btn.click(fn=load_gallery, inputs=[gal_label, gal_limit, gal_side], outputs=[gallery])
        gallery.value = load_gallery("ALL", 24, "back")


# Schritt 8: Server starten (Handy im gleichen WLAN)
def guess_local_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        s.connect(("10.255.255.255", 1))
        ip = s.getsockname()[0]
    except Exception:
        ip = "127.0.0.1"
    finally:
        s.close()
    return ip

def get_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

local_ip = guess_local_ip()
port = get_free_port()

print(f"👉 Öffne am Handy (gleiches WLAN): http://{local_ip}:{port}")
app.launch(server_name="0.0.0.0", server_port=port, share=False)



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ Schema OK (pokemon_card_samples_fb). Counts: {'EX': 33, 'GD': 30, 'LP': 30, 'NM': 31, 'PL': 28, 'PO': 25, 'TOTAL': 177}
👉 Öffne am Handy (gleiches WLAN): http://192.168.178.49:53102
* Running on local URL:  http://0.0.0.0:53102
* To create a public link, set `share=True` in `launch()`.
